In [1]:
from pathlib import Path
import os

STUDENT = 'yoric'

OUTLIERS_PATH = Path(r'C:\Users\yoric\ML4QS\ML4QS-GROUP99-Michael-FeatureEng\Python3Code\outliers2')
INTERMEDIATE_PATH = Path(r'C:\Users\yoric\ML4QS\ML4QS-GROUP99-Michael-FeatureEng\Python3Code\intermediate_datafiles')

os.chdir(r'C:\Users\yoric\ML4QS\ML4QS-GROUP99-Michael-FeatureEng\Python3Code')

EXPERIMENT_DIR = 'ML4QS-Vehicle-2'

from util.VisualizeDataset import VisualizeDataset
from Visualiser import Visualiser as Viz
from outlier_detector import OutlierDetector
from custom_imputer import CustomImputer
from util.util import ignore_actual_time, read_parquet, write_parquet
from DataLoader import PhyboxDatasetLoader

In [2]:
# Load intermediate df
intermediate_df = read_parquet(INTERMEDIATE_PATH / 'ML4QS_combined_results_example.parquet')

In [3]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

def calculate_magnitude(df, prefix):
    #Calculate magnitude for 3-axis sensor data: √(x² + y² + z²)
    x_col = f"{prefix}_X"
    y_col = f"{prefix}_Y" 
    z_col = f"{prefix}_Z"
    return np.sqrt(df[x_col]**2 + df[y_col]**2 + df[z_col]**2)

#Literature mentioned a window size of 120 is good for aggregating temporal features.
#Howevr we will have to see if our data is suited for this.
def temporal_aggregation(series, window_size=120, agg_functions=['mean', 'median', 'min', 'max', 'std']):
    result = pd.DataFrame()
    for func in agg_functions:
        if func == 'std':
            result[f"{series.name}_{func}"] = series.rolling(
                window=window_size, min_periods=1
            ).std().fillna(0)
        else:
            result[f"{series.name}_{func}"] = series.rolling(
                window=window_size, min_periods=1
            ).agg(func)
    return result

#I think phone_direction is where your phone is pointing, just like a google maps arrow
#This makes sense ecause the values seem to go up to 360
#So i define a "direction change" as 10degree or more
def calculate_direction_changes(direction_series, threshold=10):
    direction_diff = direction_series.diff().fillna(0)
    direction_diff = np.where(direction_diff > 180, direction_diff - 360, direction_diff)
    direction_diff = np.where(direction_diff < -180, direction_diff + 360, direction_diff)
    significant_changes = np.abs(direction_diff) > threshold
    return pd.Series(significant_changes.astype(int), index=direction_series.index)


def calculate_acceleration_switches(acceleration_magnitude, threshold=0.1):
    acc_diff = acceleration_magnitude.diff().fillna(0)
    acc_state = np.where(acc_diff > threshold, 1,  
                       np.where(acc_diff < -threshold, -1, 0))  
    switches = np.abs(np.diff(acc_state, prepend=acc_state[0])) > 0
    return pd.Series(switches.astype(int), index=acceleration_magnitude.index)

def process_transportation_data(df, window_size=120):
    
    # Group by session ID and process each session
    processed_sessions = []
    
    for session_id, session_data in df.groupby('id'):
        session_df = session_data.copy()
        
        # Calculate magnitudes
        session_df['acc_phone_magnitude'] = calculate_magnitude(session_df, 'acc_phone')
        session_df['lin_acc_phone_magnitude'] = calculate_magnitude(session_df, 'lin_acc_phone') 
        session_df['gyr_phone_magnitude'] = calculate_magnitude(session_df, 'gyr_phone')
        
        # Calculate frequency-based features
        session_df['direction_changes'] = calculate_direction_changes(session_df['location_phone_Direction'])
        session_df['acc_switches'] = calculate_acceleration_switches(session_df['acc_phone_magnitude'])
        session_df['lin_acc_switches'] = calculate_acceleration_switches(session_df['lin_acc_phone_magnitude'])
        session_df['rotation_switches'] = calculate_acceleration_switches(session_df['gyr_phone_magnitude'])
        
        # Apply temporal aggregation to all relevant features
        features_to_aggregate = [
            'acc_phone_X', 'acc_phone_Y', 'acc_phone_Z',
            'lin_acc_phone_X', 'lin_acc_phone_Y', 'lin_acc_phone_Z',
            'gyr_phone_X', 'gyr_phone_Y', 'gyr_phone_Z',
            'location_phone_Velocity', 'location_phone_Direction',
            'location_phone_Horizontal Accuracy', 'location_phone_Vertical Accuracy',
            'acc_phone_magnitude', 'lin_acc_phone_magnitude', 'gyr_phone_magnitude',
            'direction_changes', 'acc_switches', 'lin_acc_switches', 'rotation_switches'
        ]
        
        # Apply temporal aggregation
        for feature in features_to_aggregate:
            if feature in session_df.columns:
                aggregated = temporal_aggregation(session_df[feature], window_size)
                session_df = pd.concat([session_df, aggregated], axis=1)
        
        # Calculate session-level aggregates
        session_direction_change_rate = session_df['direction_changes'].mean()
        session_acc_switch_rate = session_df['acc_switches'].mean()
        session_lin_acc_switch_rate = session_df['lin_acc_switches'].mean()
        session_rotation_switch_rate = session_df['rotation_switches'].mean()
        session_avg_velocity = session_df['location_phone_Velocity'].mean()
        session_velocity_std = session_df['location_phone_Velocity'].std()
        
        # Add session-level features as columns to all rows in this session
        session_df['session_direction_change_rate'] = session_direction_change_rate
        session_df['session_acc_switch_rate'] = session_acc_switch_rate
        session_df['session_lin_acc_switch_rate'] = session_lin_acc_switch_rate
        session_df['session_rotation_switch_rate'] = session_rotation_switch_rate
        session_df['session_avg_velocity'] = session_avg_velocity
        session_df['session_velocity_std'] = session_velocity_std
        
        processed_sessions.append(session_df)
    
    # Concatenate all sessions back into one dataframe
    result_df = pd.concat(processed_sessions, ignore_index=True)
    
    return result_df

session_features = process_transportation_data(intermediate_df, window_size=120)

session_features.to_csv('C:/Users/yoric/ML4QS/session_features.csv', index=False)

In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

# -------------------------------
# Load data and inspect columns
# -------------------------------
df = pd.read_parquet(INTERMEDIATE_PATH / 'ML4QS_combined_results_example.parquet')

# Debug: Check what columns exist
print("Available columns:")
print(df.columns.tolist())
print(f"\nDataFrame shape: {df.shape}")
print(f"\nFirst few rows:")
print(df.head())

# Look for potential timestamp/time columns
time_like_columns = [col for col in df.columns if any(keyword in col.lower() for keyword in ['time', 'date', 'stamp'])]
print(f"\nPotential time-related columns: {time_like_columns}")

# -------------------------------
# Create target label
# -------------------------------
label_columns = ['labeltram', 'labeltrain', 'labelwalking', 'labelmetro', 'labelbus', 'labelcar', 'labelbike']
transportation_modes = ['tram', 'train', 'walking', 'metro', 'bus', 'car', 'bike']

def create_target_label(row):
    for i, col in enumerate(label_columns):
        if row.get(col, 0) == 1:
            return transportation_modes[i]
    return 'unknown'

df['transport_mode'] = df.apply(create_target_label, axis=1)
df = df[df['transport_mode'] != 'unknown'] 

print(f"\nAfter filtering unknown transport modes: {df.shape[0]} samples")
print(f"Class distribution:\n{df['transport_mode'].value_counts()}")

# -------------------------------
# Feature selection
# -------------------------------
datetime_cols = df.select_dtypes(include='datetime64').columns.tolist()
print(f"\nDateTime columns found: {datetime_cols}")

# Find the correct timestamp column
timestamp_col = None
possible_timestamp_cols = ['timestamp', 'time', 'datetime', 'date_time', 'session_time']

for col in possible_timestamp_cols:
    if col in df.columns:
        timestamp_col = col
        break

# If no standard timestamp column found, look for time-like columns
if timestamp_col is None and time_like_columns:
    timestamp_col = time_like_columns[0]  # Use the first time-like column
    print(f"\nUsing '{timestamp_col}' as timestamp column")

if timestamp_col is None:
    print("\nWARNING: No timestamp column found. Options:")
    print("1. Use random split instead of temporal split")
    print("2. Create a synthetic timestamp based on row order")
    print("3. Check if your data has a different time column name")
    
    # Option 2: Create synthetic timestamp based on session order
    print("\nCreating synthetic timestamp based on row order within each session...")
    df = df.sort_values('id').reset_index(drop=True)
    df['synthetic_timestamp'] = df.groupby('id').cumcount()
    timestamp_col = 'synthetic_timestamp'

# Ensure timestamp column is excluded from features to prevent data leakage
exclude_cols = ['id', 'transport_mode']
if timestamp_col:
    exclude_cols.append(timestamp_col)

feature_cols = [
    col for col in df.columns
    if not col.startswith('label')
    and col not in exclude_cols
]

print(f"\nUsing {len(feature_cols)} features")
print(f"Timestamp column: '{timestamp_col}'")

# -------------------------------
# Per-ID 80/20 temporal split
# -------------------------------
df = df.sort_values(by=['id', timestamp_col])

train_parts = []
test_parts = []

for session_id, group in df.groupby('id'):
    group_sorted = group.sort_values(timestamp_col)
    n = len(group_sorted)
    if n < 5:
        continue  # skip short sessions
    split_idx = int(n * 0.8)
    train_parts.append(group_sorted.iloc[:split_idx])
    test_parts.append(group_sorted.iloc[split_idx:])

if not train_parts or not test_parts:
    raise ValueError("No data after filtering short sessions. Check your session lengths.")

df_train = pd.concat(train_parts).reset_index(drop=True)
df_test = pd.concat(test_parts).reset_index(drop=True)

X_train = df_train[feature_cols]
X_test = df_test[feature_cols]
y_train = df_train['transport_mode']
y_test = df_test['transport_mode']

print(f"\nTrain size: {X_train.shape[0]} samples")
print(f"Test size: {X_test.shape[0]} samples")
print(f"Class distribution (train):\n{y_train.value_counts()}")
print(f"Class distribution (test):\n{y_test.value_counts()}")

# Check for any missing values or infinite values
print(f"\nMissing values in train features: {X_train.isnull().sum().sum()}")
print(f"Infinite values in train features: {np.isinf(X_train.select_dtypes(include=[np.number])).sum().sum()}")

# -------------------------------
# Preprocessing
# -------------------------------
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

y_train_cat = pd.Categorical(y_train)
y_test_cat = pd.Categorical(y_test, categories=y_train_cat.categories) 

y_train_encoded = y_train_cat.codes
y_test_encoded = y_test_cat.codes

print(f"\nEncoded classes: {dict(enumerate(y_train_cat.categories))}")

# -------------------------------
# Train XGBoost
# -------------------------------
dtrain = xgb.DMatrix(X_train_scaled, label=y_train_encoded)
dtest = xgb.DMatrix(X_test_scaled, label=y_test_encoded)

params = {
    'objective': 'multi:softprob',
    'num_class': len(y_train_cat.categories),
    'max_depth': 6,
    'learning_rate': 0.01,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'random_state': 42,
    'eval_metric': 'mlogloss'
}

print("\nTraining XGBoost model...")
model = xgb.train(
    params,
    dtrain,
    num_boost_round=100,
    evals=[(dtrain, 'train'), (dtest, 'eval')],
    early_stopping_rounds=50,
    verbose_eval=100
)

# -------------------------------
# Evaluation
# -------------------------------
y_pred_proba = model.predict(dtest)
y_pred = np.argmax(y_pred_proba, axis=1)

y_pred_labels = [y_train_cat.categories[i] for i in y_pred]

accuracy = accuracy_score(y_test, y_pred_labels)
print(f"\nTest Accuracy: {accuracy:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_labels))

plt.figure(figsize=(10, 8))
cm = confusion_matrix(y_test, y_pred_labels, labels=y_train_cat.categories)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=y_train_cat.categories, yticklabels=y_train_cat.categories)
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.tight_layout()
plt.show()

# -------------------------------
# Feature Importance
# -------------------------------
feature_importance = model.get_score(importance_type='weight')
importance_df = pd.DataFrame({
    'feature': feature_cols,
    'importance': [feature_importance.get(f'f{i}', 0) for i in range(len(feature_cols))]
}).sort_values('importance', ascending=False)

plt.figure(figsize=(12, 8))
sns.barplot(data=importance_df.head(20), x='importance', y='feature')
plt.title('Top 20 Most Important Features')
plt.xlabel('Feature Importance (Weight)')
plt.tight_layout()
plt.show()

print("\nMost Important Features:")
print(importance_df.head(10))

Available columns:
['id', 'timestamp', 'acc_phone_X', 'acc_phone_Y', 'acc_phone_Z', 'lin_acc_phone_X', 'lin_acc_phone_Y', 'lin_acc_phone_Z', 'gyr_phone_X', 'gyr_phone_Y', 'gyr_phone_Z', 'location_phone_Latitude', 'location_phone_Longitude', 'location_phone_Height', 'location_phone_Velocity', 'location_phone_Direction', 'location_phone_Horizontal Accuracy', 'location_phone_Vertical Accuracy', 'mag_phone_X', 'mag_phone_Y', 'mag_phone_Z', 'proximity_phone_Distance', 'labeltram', 'labeltrain', 'labelwalking', 'labelmetro', 'labelbus', 'labelcar', 'labelbike']

DataFrame shape: (53043, 29)

First few rows:
   id               timestamp  acc_phone_X  acc_phone_Y  acc_phone_Z  \
0   0 2025-06-04 11:41:49.154     0.595657     3.355480     9.099531   
1   0 2025-06-04 11:41:49.404     0.142101     3.311633     9.272894   
2   0 2025-06-04 11:41:49.654    -0.165194     3.236582     9.294871   
3   0 2025-06-04 11:41:49.904    -0.393992     3.161564     9.303603   
4   0 2025-06-04 11:41:50.154  

In [13]:
# Debug script to identify potential data leakage issues

import pandas as pd
import numpy as np

# Load the data again for debugging
df = pd.read_csv("/Users/yoric/ML4QS/session_features.csv")

print("=== DATA LEAKAGE DEBUGGING ===\n")

# 1. Check for potential leakage in feature names
print("1. CHECKING FEATURE NAMES FOR SUSPICIOUS PATTERNS:")
print("-" * 50)

suspicious_keywords = ['label', 'target', 'class', 'mode', 'transport', 'activity', 'ground_truth']
feature_cols = [col for col in df.columns if not col.startswith('label') 
                and col not in ['id', 'transport_mode']]

suspicious_features = []
for feature in feature_cols:
    for keyword in suspicious_keywords:
        if keyword in feature.lower():
            suspicious_features.append(feature)
            break

if suspicious_features:
    print(f"{suspicious_features}")
else:
    print("No obviously suspicious feature names found")

# 2. Check train-test overlap by session IDs
print(f"\n2. CHECKING TRAIN-TEST SESSION OVERLAP:")
print("-" * 50)

# Recreate the split to check overlap
label_columns = ['labeltram', 'labeltrain', 'labelwalking', 'labelmetro', 'labelbus', 'labelcar', 'labelbike']
transportation_modes = ['tram', 'train', 'walking', 'metro', 'bus', 'car', 'bike']

def create_target_label(row):
    for i, col in enumerate(label_columns):
        if row.get(col, 0) == 1:
            return transportation_modes[i]
    return 'unknown'

df['transport_mode'] = df.apply(create_target_label, axis=1)
df = df[df['transport_mode'] != 'unknown']

# Find timestamp column
timestamp_col = None
possible_timestamp_cols = ['timestamp', 'time', 'datetime', 'date_time', 'session_time']
time_like_columns = [col for col in df.columns if any(keyword in col.lower() for keyword in ['time', 'date', 'stamp'])]

for col in possible_timestamp_cols:
    if col in df.columns:
        timestamp_col = col
        break

if timestamp_col is None and time_like_columns:
    timestamp_col = time_like_columns[0]

if timestamp_col is None:
    df = df.sort_values('id').reset_index(drop=True)
    df['synthetic_timestamp'] = df.groupby('id').cumcount()
    timestamp_col = 'synthetic_timestamp'

# Recreate the split
df = df.sort_values(by=['id', timestamp_col])
train_parts = []
test_parts = []

for session_id, group in df.groupby('id'):
    group_sorted = group.sort_values(timestamp_col)
    n = len(group_sorted)
    if n < 5:
        continue
    split_idx = int(n * 0.8)
    train_parts.append(group_sorted.iloc[:split_idx])
    test_parts.append(group_sorted.iloc[split_idx:])

df_train = pd.concat(train_parts).reset_index(drop=True)
df_test = pd.concat(test_parts).reset_index(drop=True)

train_sessions = set(df_train['id'].unique())
test_sessions = set(df_test['id'].unique())
session_overlap = train_sessions.intersection(test_sessions)

print(f"Train sessions: {len(train_sessions)}")
print(f"Test sessions: {len(test_sessions)}")
print(f": {len(session_overlap)}")
print(f"Overlap percentage: {len(session_overlap)/len(train_sessions)*100:.1f}%")

if len(session_overlap) > 0:
    print(" Same sessions appear in both train and test!")
    print("This causes severe data leakage!")

# 3. Check feature distributions between train and test
print(f"\n3. CHECKING FEATURE DISTRIBUTIONS:")
print("-" * 50)

feature_cols = [col for col in df.columns if not col.startswith('label') 
                and col not in ['id', 'transport_mode', timestamp_col]]

X_train = df_train[feature_cols]
X_test = df_test[feature_cols]

# Check for identical distributions (another leakage sign)
suspicious_identical_features = []
for col in feature_cols[:10]:  # Check first 10 features
    if col in X_train.columns and col in X_test.columns:
        train_vals = X_train[col].dropna()
        test_vals = X_test[col].dropna()
        
        if len(train_vals) > 0 and len(test_vals) > 0:
            # Check if distributions are suspiciously similar
            train_mean = train_vals.mean()
            test_mean = test_vals.mean()
            train_std = train_vals.std()
            test_std = test_vals.std()
            
            if (abs(train_mean - test_mean) < 0.001 and 
                abs(train_std - test_std) < 0.001):
                suspicious_identical_features.append(col)

if suspicious_identical_features:
    print(f"Features with identical distributions: {suspicious_identical_features[:5]}")

# 4. Check class distribution per session
print(f"\n4. CHECKING CLASS DISTRIBUTIONS PER SESSION:")
print("-" * 50)

session_class_counts = df.groupby(['id', 'transport_mode']).size().unstack(fill_value=0)
pure_sessions = []

for session_id in session_class_counts.index:
    non_zero_classes = (session_class_counts.loc[session_id] > 0).sum()
    if non_zero_classes == 1:  # Only one transport mode per session
        pure_sessions.append(session_id)

print(f"Sessions with only one transport mode: {len(pure_sessions)}/{len(session_class_counts)}")
print(f"Percentage of 'pure' sessions: {len(pure_sessions)/len(session_class_counts)*100:.1f}%")

if len(pure_sessions) / len(session_class_counts) > 0.8:
    print("Most sessions have only one transport mode!")
    print("This makes the temporal split essentially a session-level split,")
    print("which might be too easy for the model.")

# 5. Check for perfect separability features
print(f"\n5. CHECKING FOR PERFECT SEPARABILITY:")
print("-" * 50)

from sklearn.tree import DecisionTreeClassifier

# Quick check if a simple decision tree can achieve perfect accuracy
dt = DecisionTreeClassifier(max_depth=3, random_state=42)
dt.fit(X_train.fillna(0), df_train['transport_mode'])
dt_score = dt.score(X_test.fillna(0), df_test['transport_mode'])

print(f"Simple decision tree accuracy: {dt_score:.4f}")
if dt_score > 0.95:
    print("Even a simple decision tree gets >95% accuracy!")
    print("This suggests the problem might be too easy or has leakage.")

# 6. Show sample of the most important features
print(f"\n6. EXAMINING TOP FEATURES:")
print("-" * 50)

top_features = ['session_avg_velocity', 'session_acc_switch_rate', 'session_velocity_std']
for feature in top_features:
    if feature in df.columns:
        print(f"\n{feature}:")
        feature_by_mode = df.groupby('transport_mode')[feature].agg(['mean', 'std', 'min', 'max'])
        print(feature_by_mode.round(3))

print(f"\n=== RECOMMENDATIONS ===")
print("1. If sessions overlap between train/test: Use proper session-level split")
print("2. If features are too discriminative: Check feature engineering process")
print("3. If sessions are 'pure': Consider if this reflects real-world usage")
print("4. Validate with cross-validation across different sessions")
print("5. Test on completely new data from different time periods/users")

=== DATA LEAKAGE DEBUGGING ===

1. CHECKING FEATURE NAMES FOR SUSPICIOUS PATTERNS:
--------------------------------------------------
No obviously suspicious feature names found

2. CHECKING TRAIN-TEST SESSION OVERLAP:
--------------------------------------------------
Train sessions: 27
Test sessions: 27
OVERLAPPING: 27
Overlap percentage: 100.0%
Same sessions appear in both train and test!
This causes severe data leakage!

3. CHECKING FEATURE DISTRIBUTIONS:
--------------------------------------------------

4. CHECKING CLASS DISTRIBUTIONS PER SESSION:
--------------------------------------------------
Sessions with only one transport mode: 27/27
Percentage of 'pure' sessions: 100.0%
Most sessions have only one transport mode!
This makes the temporal split essentially a session-level split,
which might be too easy for the model.

5. CHECKING FOR PERFECT SEPARABILITY:
--------------------------------------------------
Simple decision tree accuracy: 0.7874

6. EXAMINING TOP FEATURES:
-

In [ ]:
#here we can see what the predictions on the test set actually were
predicted_counts = pd.Series(y_pred_labels).value_counts()
print("Frequency of predicted classes:")
print(predicted_counts)

In [ ]:
#Labels were wrong first should be ok now
print(y_test.head())
print(y_pred_labels[:5])



In [ ]:
import numpy as np
# For a few samples
for i in range(5):
    print(f"Sample {i} predicted probabilities: {y_pred_proba[i]}")
    print(f"Predicted class: {y_pred_labels[i]}, True class: {y_test.iloc[i]}")